In [62]:
import pandas as pd
import matplotlib.pylab as plt
import matplotlib as mpl
import numpy as np
from matplotlib.patches import Circle, Rectangle, Arc
import seaborn as sns
plt.style.use('seaborn-dark-palette')
mypal = plt.rcParams['axes.prop_cycle'].by_key()['color'] # Grab the color pal
import os
import gc
from collections import Counter

In [63]:
cd ../data

/Users/johnwalecka/Desktop/Springboard/capstone_project_1/capstone_project_1_repo/data


In [ ]:
events = pd.read_csv('events_2015.csv', index_col='EventID')
team_ids = pd.read_csv('teams.csv', index_col='TeamID')
games = pd.read_csv('MRegularSeasonCompactResults.csv', index_col='Season').loc['2015']
rankings = pd.read_csv('MMasseyOrdinals.csv', index_col='Season').loc['2015']

In [ ]:
games.index = range(len(games))
final_rankings = rankings[(rankings['RankingDayNum'] == 133) & (rankings['SystemName'] == '7OT')].pivot_table(index='TeamID', values='OrdinalRank')
final_rankings.head()

In [ ]:
for index, row in games.iterrows():
    games.loc[index, 'WTeamRank'] = int(final_rankings.loc[games.iloc[index, 1]])
    games.loc[index, 'LTeamRank'] = int(final_rankings.loc[games.iloc[index, 3]])

In [ ]:
basic_stats_dist = final_rankings
for index, row in basic_stats_dist.iterrows():
    basic_stats_dist.loc[index, 'wins'] = len(games[games['WTeamID'] == index])
    basic_stats_dist.loc[index, 'losses'] = len(games[games['LTeamID'] == index])
    basic_stats_dist.loc[index, 'win_pct'] = basic_stats_dist.loc[index, 'wins']/(basic_stats_dist.loc[index, 'wins'] + basic_stats_dist.loc[index, 'losses'])
    basic_stats_dist.loc[index, 'win_top50'] = len(games[(games['WTeamID'] == index) & (games['LTeamRank'] <= 50)])


In [ ]:
events['counter'] = 1

In [ ]:
team_shot_counters = dict()
for index in team_ids.index:
    if ((index in events.WTeamID.values) or (index in events.LTeamID.values)):
        shot_list = list(events[events.EventTeamID == index]['EventType'])
        dist_dict = Counter(shot_list)
        team_shot_counters[index] = dist_dict
print(team_shot_counters.get(1101))

In [ ]:
shot_dist = final_rankings
inf = 1000000
for team_id, shot_counter in team_shot_counters.items():
    made_1 = shot_counter.get('made1', 0)
    miss_1 = shot_counter.get('miss1', 0)
    made_2 = shot_counter.get('made2', 0)
    miss_2 = shot_counter.get('miss2', 0)
    made_3 = shot_counter.get('made3', 0)
    miss_3 = shot_counter.get('miss3', 0)
    
    shot_dist.loc[team_id, 'ones_attempted_pct'] = (miss_1 + made_1)/total_attempted
    shot_dist.loc[team_id, 'ones_made_pct'] = made_1/total_made
    shot_dist.loc[team_id, 'twos_attempted_pct'] = (miss_2 + made_2)/total_attempted
    shot_dist.loc[team_id, 'twos_made_pct'] = made_2/total_made
    shot_dist.loc[team_id, 'threes_attempted_pct'] = (miss_3 + made_3)/total_attempted
    shot_dist.loc[team_id, 'threes_made_pct'] = made_3/total_made
    
shot_dist.sort_values(by='OrdinalRank', ascending=True)
    

In [ ]:
total_attempted = shot_counter.get('miss1') + shot_counter.get('made1') + shot_counter.get('miss2') + shot_counter.get('made2') + shot_counter.get('miss3') + shot_counter.get('made3')
    total_made = shot_counter.get('made1') + shot_counter.get('made2') + shot_counter.get('made3')
    shot_dist.loc[index, 'fts_attempted_pct'] = (shot_counter.get('miss1') + shot_counter.get('made1'))/total_attempted
    shot_dist.loc[index, 'fts_made_pct'] = shot_counter.get('made1')/total_made
    shot_dist.loc[index, 'twos_attempted_pct'] = (shot_counter.get('miss2') + shot_counter.get('made2'))/total_attempted
    shot_dist.loc[index, 'twos_made_pct'] = shot_counter.get('made2')/total_made
    shot_dist.loc[index, 'threes_attempted_pct'] = (shot_counter.get('miss3') + shot_counter.get('made3'))/total_attempted
    shot_dist.loc[index, 'threes_made_pct'] = shot_counter.get('made3')/total_made
shot_dist.head()

In [ ]:
events.groupby('EventType')['counter'].sum().sort_values(ascending=False).plot(kind='bar',
          figsize=(15, 5),
         color=mypal[2],
         title='Event Type Frequency (Mens)')
plt.xticks(rotation=0)
plt.show()